In [3]:
!pip install ultralytics
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.5 MB/s eta 0:00:00


In [4]:
from ultralytics import YOLO

# Load your trained YOLOv8 model
# Replace 'best.pt' with the path to your model file if it's different
model = YOLO('best.pt')

# Export the model to ONNX format
# The exported ONNX model will be saved in the same directory as your model file
model.export(format='onnx')

print("Model exported to ONNX using ultralytics export function.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.212 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (AMD EPYC 7B12)
Model summary (fused): 112 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs

PyTorch: starting from 'best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (130.4 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.71'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 2.4s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.71...
ONNX: export success ✅ 18.8s, saved as 'best.onnx' (260.1 MB)

Export complete (26.

In [5]:
import onnxruntime
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

# Load the ONNX model
onnx_model_path = 'best.onnx'
ort_session = onnxruntime.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

# Load and preprocess the image
image_path = '/content/36455_4751560.jpg'
img = Image.open(image_path).convert('RGB') # YOLOv8 expects 3 channels

# The model was trained with an input size of 640x640, so we resize the image
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

input_tensor = transform(img)

# ONNX Runtime expects numpy arrays, so convert the input tensor
input_numpy = input_tensor.unsqueeze(0).numpy() # Add batch dimension and convert to numpy

# Get input name from ONNX model
input_name = ort_session.get_inputs()[0].name

# Run inference with ONNX Runtime
onnxruntime_input = {input_name: input_numpy}
results_onnx = ort_session.run(None, onnxruntime_input)

print("Inference with ONNX Runtime completed.")
# The output format of ONNX Runtime might be different from PyTorch's,
# you'll likely need to post-process results_onnx to interpret detections.
print(results_onnx)

Inference with ONNX Runtime completed.
[array([[[     15.158,       20.97,      26.594, ...,      520.43,      559.43,      626.25],
        [     23.222,      19.643,      13.583, ...,      582.18,      580.32,      606.13],
        [     30.388,      40.937,      52.213, ...,      237.06,      159.98,      188.47],
        [     46.662,      39.313,      27.193, ...,      117.63,      144.51,       158.6],
        [ 7.4506e-07,  2.0862e-07,  2.3842e-07, ...,  8.9407e-08,           0,           0]]], dtype=float32)]
